In [33]:
##How to perform active machine learning

# Normal libraries

from pandas import read_csv
import pandas as pd
import numpy as np
import datetime

# Library to build a model using keras that is build on tensorflow
# (a library on how to handle tensor and their function)
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

# Library to build generic models

from scipy.optimize import curve_fit


# Libraries about the learning process of the actual AI
from keras.callbacks import ModelCheckpoint
# from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Useful libraries
from matplotlib import pyplot as plt
import math

In [34]:
Sensors = pd.read_csv("../../../Downloads/appa2_timeline.csv", sep=",")
Sensors.set_index("channel", inplace=True)
Sensors = Sensors.applymap(lambda x: x.replace("*", ""))

df = read_csv("../../DataSet_FBK/ViaBolz.csv", sep=",", parse_dates=["ts"])
df.set_index("ts", inplace=True)

change = {}
for i in range(1, 9):
    change[f"S{str(i)}_R1"] = f"T{str(i)}"
    change[f"S{str(i)}_R2"] = f"R{str(i)}"
    change[f"S{str(i)}_Voltage"] = f"V{str(i)}"
df = df.rename(columns=change)


Test = {}
maxim = len(Sensors.columns)
for i, col in enumerate(Sensors.columns):
    materials = []
    if i == maxim - 1:
        for j, material in enumerate(Sensors[col]):
            f = materials.count(material) + 1
            materials.append(material)
            if list(Test.keys()).count(material + str(f)) == 1:
                Test[material + str(f)] = pd.concat(
                    [
                        Test[material + str(f)],
                        df[col:][
                            ["R" + str(j + 1), "T" + str(j + 1), "V" + str(j + 1)]
                        ].rename(
                            columns={
                                "R" + str(j + 1): "R",
                                "T" + str(j + 1): "HT",
                                "V" + str(j + 1): "V",
                            }
                        ),
                    ]
                )
            else:
                Test[material + str(f)] = df[col:][
                    ["R" + str(j + 1), "T" + str(j + 1), "V" + str(j + 1)]
                ].rename(
                    columns={
                        "R" + str(j + 1): "R",
                        "T" + str(j + 1): "HT",
                        "V" + str(j + 1): "V",
                    }
                )
    else:
        for j, material in enumerate(Sensors[col]):
            f = materials.count(material) + 1
            materials.append(material)
            if list(Test.keys()).count(material + str(f)) == 1:
                Test[material + str(f)] = pd.concat(
                    [
                        Test[material + str(f)],
                        df[col : Sensors.columns[i + 1]][
                            ["R" + str(j + 1), "T" + str(j + 1), "V" + str(j + 1)]
                        ].rename(
                            columns={
                                "R" + str(j + 1): "R",
                                "T" + str(j + 1): "HT",
                                "V" + str(j + 1): "V",
                            }
                        ),
                    ]
                )
            else:
                Test[material + str(f)] = df[col : Sensors.columns[i + 1]][
                    ["R" + str(j + 1), "T" + str(j + 1), "V" + str(j + 1)]
                ].rename(
                    columns={
                        "R" + str(j + 1): "R",
                        "T" + str(j + 1): "HT",
                        "V" + str(j + 1): "V",
                    }
                )

df8 = Test["LaFeO31"]

df_appa = read_csv("../../DataSet_APPA/APPA2.csv")
df_appa
df_appa.Time = pd.to_datetime(df_appa.Time)
timezone_offset = datetime.timedelta(hours=2)
timezone = datetime.timezone(timezone_offset)

df_appa.Time = df_appa.Time.apply(lambda x: x.replace(tzinfo=timezone))
df_appa.rename(columns = {"Time":"ts"}, inplace = True)

In [40]:


df8 = df8.resample("1H").mean()
df_tot = df8.reset_index().merge(df_appa)
df_tot.set_index("ts",inplace=True)
df_tot = df_tot.sort_index().dropna(how="all")
df_tot = df_tot.dropna(how="all")

AttributeError: 'DataFrame' object has no attribute 'Time'

In [36]:
##Example of curve fit


##Define a model
def f_model(x, a, b, c, d, e):
    Sig, T, RH, AH = x
    return a * Sig**b + d * T + e + c * AH


##df8 is an example of a dataframe, indexed with data and with columns name not really informative

##You are training here from a start to end the slicing of a database
(
    popt,
    pcov,
) = curve_fit(  # popt are the optimal values, pcov is the covariance matrix of the optimized values.
    f=f_model,  # model function
    xdata=(
        df8.sort_index().loc["2021-01-01":"2021-02-1"]["V"].values,
        df8.sort_index().loc["2021-01-01":"2021-02-1"]["R"].values,
        df8.sort_index().loc["2021-01-01":"2021-02-1"]["HT"].values,
    ),  # x data
    ydata=df_tot["2021-01-01":"2022-01-1"]["PM10"].values,  # y data
    p0=(4, 1, 1, 1, 1, 1, 1),  # initial value of the parameters
    maxfev=10000,  # Number of times in which you repeat the training, close to epochs
)
a_opt, b_opt, c_opt, d_opt, e_opt, f_opt, g_opt = popt
df8["model 1"] = f_model(
    (df8["V"].values, df8["R"].values, df8["HT"].values),
    a_opt,
    b_opt,
    c_opt,
    d_opt,
    e_opt,
    f_opt,
    g_opt,
)  # Here you are predicting the value of your complete samples

ValueError: array must not contain infs or NaNs

In [ ]:
##Example of Scaler, matrices or something like these

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_square_error

##Place whatever dataframe you want
X = df8.drop([#Place here the values you want to predict], axis = 1)

y = df8[[#Place here the values you want to predict]].copy()

INPUT_DIM = len(X.columns())
OUTPUT_DIM = len(y.columns())

In [ ]:
# Split training and test, remember that you may want to not shuffle data while dividing them
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=True
)
# You perform a standard scaler (or minmax scaler) to make the data more usable by the machine learning
# model
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

scaler = StandardScaler()
scaler.fit(y_train)

y_train_s = scaler.transform(y_train)
y_test_s = pd.DataFrame(data=scaler.transform(y_test), columns=y_test.columns)

In [ ]:
# define the model
# Experiment with deeper and wider networks
model = Sequential()
# By Sequential it means that the layers are connected one after the one before so that you have
# a defined flow of information


# Input layer, Dense layer meaning that the layer is connected to all the neurons to the layer before
# kernel_initializer is the how the starting values of the parameters of your neural network (NN)
# input_dim is the number of features of the dataset that you are feeding to your NN
# activation function is how you introduce non linearity in your model.

model.add(
    Dense(64, kernel_initializer="normal", input_dim=INPUT_DIM, activation="selu")
)

# Hidden layers:
# Dropout layer is just a layer where you drops, for example the 20% of the links between the two layers
# that it is placed between
model.add(Dropout(0.2))

model.add(Dense(128, kernel_initializer="normal", activation="selu"))
model.add(Dropout(0.2))
model.add(Dense(64, kernel_initializer="normal", activation="selu"))
model.add(Dropout(0.2))

model.add(Dense(16, kernel_initializer="normal", activation="selu"))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(OUTPUT_DIM, kernel_initializer="normal", activation="linear"))

# with add you can add a new layer, with pop you remove the last layer you have placed there


# You are compiling your model, so from now on you can't change its structure. Here you specify its loss function,
# the function that you want to minimize, the optimizer that is how you want to optimize the parameters,
# the metrics that is another value that you can use to measure your model but you won't use it to optimize
# the model
model.compile(loss=a2, optimizer="adam", metrics=[a2])
# You are showing how your NN is formed
model.summary()

##Here you are defining something that is not necessary. You are defining condition by which you are saving
# definite models. For example in that case you are specifing where do you want to place them, you are searching
# the model where the value of the loss function over the validation set (data not seen by the model during
# the training) is minimum. The weight of the "best model" by that value is saved there.
checkpoint_name = "./tmp/checkpoint"
checkpoint = ModelCheckpoint(
    checkpoint_name, monitor="val_loss", verbose=1, save_best_only=True, mode="min"
)
callbacks_list = [checkpoint]
# In the history you collect the important information that happens during the fit, value of loss function,
# metrics and so on in one variable.
# During the fit  the parameters of the NN are updated. You are specifing the training data, X and y, and either
# you give the fit function the validation data or you give them a percentage as the validation_split, in this case 20%
# after that you are specifing the number of epochs, meaning how many time the model has seen the same data,
# and then the callbacks that you want to use. Notice that you can have more than one callback.
# There are more parameters that you can implement, so take a look at them
history = model.fit(
    X_train_scaled, y_train_s, validation_split=0.2, epochs=300, callbacks=[checkpoint]
)
# After performing the training you are uploading the best model that you have saved in the callback, updating
# the value of your parameters
model.load_weights(checkpoint_name)

In [ ]:
# plot the training and validation accuracy and loss at each epoch
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, "y", label="Training loss")
plt.plot(epochs, val_loss, "r", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()


acc = history.history["mean_squared_error"]
val_acc = history.history["val_mean_squared_error"]
plt.plot(epochs, acc, "y", label="Training MAE")
plt.plot(epochs, val_acc, "r", label="Validation MAE")
plt.title("Training and validation MAE")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

############################################
# Predict on test data
predictions = model.predict(X_test_scaled)

predictions = scaler.inverse_transform(predictions)

y_test_ssaved = y_test_s
y_test_s = y_test
print("Predicted values are: ", predictions)
print("Real values are: ", y_test_s)

##############################################
# Plot and compare prediction and real value
t = np.arange(0, predictions.size, 1)
plt.scatter(predictions, y_test_s, label="Prediction")
plt.scatter(y_test_s, y_test_s, label="Real Value")
plt.title("Prediction")
plt.xlabel("Prediction")
plt.ylabel("Real Value")
plt.legend()
plt.show()

In [ ]:
##Example of RandomForestRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error

from sklearn.metrics import r2_score


In [ ]:
##Same thing you have done before, define an X and y from a dataset if you want to try.

xtrain, xtest, ytrain, ytest=train_test_split(X, Y, shuffle = True, test_size=0.20)
scaler=MinMaxScaler()
scaler2 = MinMaxScaler()
scaler.fit(xtrain)

X_train = scaler.transform(xtrain)
X_test = scaler.transform(xtest)
scaler2.fit(ytrain.values.reshape(-1, 1))

y_train = scaler2.transform(ytrain.values.reshape(-1, 1))
y_test = scaler2.transform(ytest.values.reshape(-1, 1))

In [ ]:
rfr = RandomForestRegressor()

 
#Here are some hyperparameter of your regressor, it's a bit better and easier to look up at what the difference
#parameter does. All of this hyperparameter can be optimized using the validation data.
RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features=3, max_leaf_nodes=None,
                      max_samples=0.1, min_impurity_decrease=0.0,
                      min_samples_leaf=0.01,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False) 

In [ ]:
#fit of the RandomForestRegressor

rfr.fit(X_train, y_train.reshape(-1, 1))
#evaluation of its performance by the R-squared

score = rfr.score(X_train, y_train.reshape(-1, 1))
print("R-squared:", score) 

ypred = rfr.predict(X_test)
#evaluation of its performance by the mean_squared_error

mse = mean_squared_error(y_test, ypred)
print("MSE: ", mse)
print("RMSE: ", mse*(1/2.0)) 